In [1]:
import numpy as np
import csv
from sklearn.impute import SimpleImputer


In [2]:

with open("../Data/Raw/Asteroid_Updated.csv", "r") as file_descriptor:
    lines_reader = csv.reader(file_descriptor, delimiter=",")
    data_array = np.array(list(map(lambda line: list(map(lambda item: np.nan if len(item) == 0 else item, line)), lines_reader)))

In [3]:
with open("AuxiliaryData/Asteroid_Updated.bin", "wb") as file_descriptor:
    np.save(file_descriptor, data_array)

In [4]:
with open("AuxiliaryData/Asteroid_Updated.bin", "rb") as file_descriptor:
    data_array = np.load(file_descriptor)

In [26]:
use_number_rows = (1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 15, 17, 18, 19, 20, 21, 25, 26, 28, 29, 30)
use_string_rows = (13, 14, 23, 24, 27)

def reduce_data_array(current_data_array: np.array, use_rows: tuple, type_to_cast_to: type) -> np.array:
    reduced_array = np.array([])

    print(use_rows[0])
    reduced_array = np.hstack((reduced_array, current_data_array[1:, use_number_rows[0]].astype(type_to_cast_to)))

    for row in use_rows[1:]:
        print(row)
        reduced_array = np.vstack((reduced_array, current_data_array[1:, row].astype(type_to_cast_to)))

    return np.transpose(reduced_array)


typed_numerical_data_array = reduce_data_array(data_array, use_number_rows, float)
typed_categorical_data_array = reduce_data_array(data_array, use_string_rows, str)

header_list = [data_array[0, it] for it in use_number_rows] + [data_array[0, jt] for jt in use_string_rows]

print(header_list)

1
2
3
4
5
6
7
8
9
11
12
15
17
18
19
20
21
25
26
28
29
30
13
14
23
24
27
['a', 'e', 'i', 'om', 'w', 'q', 'ad', 'per_y', 'data_arc', 'n_obs_used', 'H', 'diameter', 'albedo', 'rot_per', 'GM', 'BV', 'UB', 'G', 'moid', 'n', 'per', 'ma', 'neo', 'pha', 'spec_B', 'spec_T', 'class']


In [32]:
diameter_col_nr = np.where(np.array(header_list) == "diameter")[0][0]

lines_with_diameter = np.logical_not(np.isnan(typed_numerical_data_array[:, diameter_col_nr]))

typed_numerical_data_with_diameter_array = typed_numerical_data_array[lines_with_diameter]
typed_categorical_data_with_diameter_array = typed_categorical_data_array[lines_with_diameter]

KeyboardInterrupt: 

In [56]:
def get_columns_with_nan_from_numeric(current_data_array: np.array) -> tuple:
    return tuple([it for it in range(current_data_array.shape[1]) if True in np.isnan(current_data_array[:, it])])

def get_columns_with_nan_from_str(current_data_array) -> tuple:
    return tuple([it for it in range(current_data_array.shape[1]) if (np.where(typed_categorical_data_with_diameter_array[:, it] == "nan"))[0].shape[0] > 0])

def get_completed_array(partial_array: np.array, nan_columns: tuple, current_imputer: SimpleImputer) -> np.array:
    completed_array = np.copy(partial_array)

    for column in nan_columns:
        completed_array[:, column] = current_imputer.fit_transform(completed_array[:, column].reshape(-1, 1))[:, 0]

    return completed_array

In [57]:
#find numerical columns that should be completed

list_of_nan_numerical_columns = get_columns_with_nan_from_numeric(typed_numerical_data_with_diameter_array)
print(list_of_nan_numerical_columns)

#complete those numerical columns with imputer

completed_typed_numerical_data_with_diameter_array = get_completed_array(typed_numerical_data_with_diameter_array,
                                                                         list_of_nan_numerical_columns,
                                                                         SimpleImputer(missing_values=np.nan, strategy="mean"))

print(get_columns_with_nan_from_numeric(completed_typed_numerical_data_with_diameter_array))

(8, 10, 12, 13, 14, 15, 16, 17)
()


In [59]:
#find categorical columns that should be completed

list_of_nan_categorical_columns = get_columns_with_nan_from_str(typed_categorical_data_with_diameter_array)
print(list_of_nan_categorical_columns)

#complete those categorical colums with most common imputer

completed_typed_categorical_data_with_diameter_array = get_completed_array(typed_categorical_data_with_diameter_array,
                                                                           list_of_nan_categorical_columns,
                                                                           SimpleImputer(missing_values="nan", strategy="most_frequent"))

print(get_columns_with_nan_from_str(completed_typed_categorical_data_with_diameter_array))

(2, 3)


ValueError: SimpleImputer does not support data with dtype <U32. Please provide either a numeric array (with a floating point or integer dtype) or categorical data represented either as an array with integer dtype or an array of string values with an object dtype.

In [ ]:
def get_categorical_matrix_from_row_and_categories(row_to_convert: np.array, categories: np.array) -> np.array:
    pass

def get_categories_from_raw(row_to_categorize: np.array) -> np.array:
    pass

def get_categorical_matrix_from_row(row_to_convert: np.array) -> np.array:
    pass

categorical_rows = []

for categorical_row in categorical_rows:
    get_categorical_matrix_from_row(data_array[categorical_row])

